In [ ]:
import cv2
import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from keras.models import Model
import numpy as np
import os
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def VGGupdated():
    img_input = Input(shape=(224, 224, 3))

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)


    # Classification block
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(2, activation='softmax')(x)

    # Create model.


    model = Model(inputs = img_input, outputs = x)


    return model

In [ ]:
vgg_model = VGGupdated()
vgg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                             

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/train'
val_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/val'
test_dir = '/content/drive/MyDrive/Data_Int/BreakHis_v1_100/output/test'

In [ ]:
#train_datagen = ImageDataGenerator(
#    rescale=1./255,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True
#)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 1455 images belonging to 2 classes.


In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 415 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 211 images belonging to 2 classes.


In [ ]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=100
)

Epoch 1/100
90/90 [==============================] - 506s 5s/step - loss: 0.7068 - accuracy: 0.6796 - val_loss: 0.6255 - val_accuracy: 0.6850
Epoch 2/100
90/90 [==============================] - 56s 616ms/step - loss: 0.6192 - accuracy: 0.6901 - val_loss: 0.6135 - val_accuracy: 0.6975
Epoch 3/100
90/90 [==============================] - 55s 612ms/step - loss: 0.6258 - accuracy: 0.6894 - val_loss: 0.6200 - val_accuracy: 0.6900
Epoch 4/100
90/90 [==============================] - 59s 656ms/step - loss: 0.6227 - accuracy: 0.6915 - val_loss: 0.6145 - val_accuracy: 0.6975
Epoch 5/100
90/90 [==============================] - 59s 651ms/step - loss: 0.6211 - accuracy: 0.6908 - val_loss: 0.6150 - val_accuracy: 0.6950
Epoch 6/100
90/90 [==============================] - 57s 625ms/step - loss: 0.6234 - accuracy: 0.6894 - val_loss: 0.6157 - val_accuracy: 0.6950
Epoch 7/100
90/90 [==============================] - 59s 660ms/step - loss: 0.6190 - accuracy: 0.6921 - val_loss: 0.6302 - val_accuracy: 0

In [ ]:
test_loss, test_acc = vgg_model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

13/13 [==============================] - 76s 6s/step - loss: 0.6212 - accuracy: 0.6875
Test accuracy: 0.6875
Test loss: 0.6211737990379333


In [ ]:
vgg_model.save('/content/drive/My Drive/vgg_model.h5')